In [1]:
import sys
sys.path.append('../../')

In [2]:
import random
random.seed(4444)
import pickle
import numpy as np
np.random.seed(5555)
import tensorflow as tf
tf.set_random_seed(6666)
import pandas as pd
from utils.dataset import DataSet, LABELS
from models.linxi import LinXiSingleModel, LinXiCompositeModel
from env import *

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
WORD_SEQS_PATH = os.path.join(CACHES_PATH, 'word_seqs/simple')

TRAIN_SEQS_PADDED_PKL = os.path.join(WORD_SEQS_PATH, 'train_seqs_padded.pkl')
VAL_SEQS_PADDED_PKL = os.path.join(WORD_SEQS_PATH, 'val_seqs_padded.pkl')
TEST_SEQS_PADDED_PKL = os.path.join(WORD_SEQS_PATH, 'test_seqs_padded.pkl')

RESULT_CSV = os.path.join(RESULTS_PATH, 'baseline.csv')

In [4]:
raw_dataset = DataSet()
train, val, test = raw_dataset.train, raw_dataset.val, raw_dataset.test

with open(TRAIN_SEQS_PADDED_PKL, 'rb') as f:
    train_seqs_padded = pickle.load(f)
    
with open(VAL_SEQS_PADDED_PKL, 'rb') as f:
    val_seqs_padded = pickle.load(f)
    
with open(TEST_SEQS_PADDED_PKL, 'rb') as f:
    test_seqs_padded = pickle.load(f)

In [5]:
train_with_seq = pd.merge(train, train_seqs_padded, on='id')
val_with_seq = pd.merge(val, val_seqs_padded, on='id')
test_with_seq = pd.merge(test, test_seqs_padded, on='id')

In [6]:
y_cols = LABELS[:2]

train_x = np.array(list(train_with_seq['word_seq']))
train_y = train_with_seq[y_cols]
val_x = np.array(list(val_with_seq['word_seq']))
val_y = val_with_seq[y_cols]
comps = [(LinXiSingleModel, {"maxlen":train_x.shape[1]})] * len(y_cols)
comp_model = LinXiCompositeModel(comps)
comp_model.fit(train_x, train_y, val = (val_x, val_y), y_cols = y_cols, seq = 'word_seq')

Train on 105000 samples, validate on 15000 samples
Epoch 1/1
105000/105000 [==============================] - 20s 192us/step - loss: 0.2772 - acc: 0.9180 - val_loss: 0.2002 - val_acc: 0.9405
The 0th grain: f1_score - 0.5965102077864759, acc - 0.9405333333333333
Train on 105000 samples, validate on 15000 samples
Epoch 1/1
105000/105000 [==============================] - 19s 179us/step - loss: 0.3710 - acc: 0.8677 - val_loss: 0.3243 - val_acc: 0.8832
The 1th grain: f1_score - 0.40867917919478014, acc - 0.8832


/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [7]:
MODEL_PRE = os.path.join(SAVED_MODELS_PATH, 'linxi', 'cnn_')
comps_1 = [(LinXiSingleModel, {'model_file': MODEL_PRE + str(i) + '.h5'}) for i in range(len(y_cols))]
comp_model_1 = LinXiCompositeModel(comps_1)

In [8]:
weights_files = [MODEL_PRE + str(i) + '.h5' for i in range(len(y_cols))]
comp_model.load_weights(weights_files)